In [6]:
! pip install pinecone-client==2.2.4 #restart the kernel after executing this cell
! sudo python -m spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 270.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [8]:
%run RAG.ipynb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
import pandas as pd
import spacy
import torch

from selfcheckgpt.modeling_selfcheck import SelfCheckNLI

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available

SelfCheck-NLI initialized to device cuda


In [13]:
chat = ChatOpenAI(temperature=0, 
                            model='gpt-3.5-turbo-0613')

conversation_openai = ConversationChain(
        llm=chat,
        memory=ConversationSummaryMemory(llm=chat),
        verbose=False
    )

nlp = spacy.load("en_core_web_sm")


# Function that returns the response from the RAG for the evaluation dataset
def get_answers(question, n_samples:int=4):
    # This function should return a list of 4 answers
    # For example:
    # return ["Answer 1", "Answer 2", "Answer 3", "Answer 4"]
    samples = []

    system_prompt, contexts = build_system_prompt(question, use_hyde=False)            
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=question
        ),
    ]
    
    for _ in range(n_samples):
        response = conversation_openai.predict(input=messages)
        samples.append(response)

    return samples

# Assuming the existence of this scoring function
def score_responses(passage, samples):
    # This function should return a list of scores for each sentence in response1
    # For example:
    # return [0.9, 0.85, 0.95]  # Example scores for each sentence in response1
    sentences = [sent.text.strip() for sent in nlp(passage).sents] # spacy sentence tokenization
    sent_scores_nli = selfcheck_nli.predict(
        sentences = sentences, # list of sentences
        sampled_passages = samples, # list of sampled passages
    )
    return sent_scores_nli

# Function to process each question
def process_questions(row):
    answers = get_answers(row['Question'])
    first_answer = answers[0]
    other_answers = answers[1:]
    scores = score_responses(first_answer, other_answers)
    median_score = pd.Series(scores).median() if len(scores) > 1 else scores[0]
    return pd.Series([first_answer, scores, median_score], index=['Response', 'Scores', 'Median Score'])


In [ ]:
# Example DataFrame with questions
data = {
    'Question': ["How can I track my order status on Rakuten?",
            "What is Rakuten's return policy for electronics?",
            "Can I change the shipping address after placing my order?",
            "What payment methods are accepted on Rakuten?",
            "Is it possible to cancel my order after it has been shipped?"]
}
df = pd.DataFrame(data)


# Apply the function to each row in the DataFrame and concatenate the results
hallucination_eval_df = pd.concat([process_questions(row) for index, row in df.iterrows()]).reset_index(drop=True)

print(hallucination_eval_df)

WARNING! Your documents have duplicate entries!  This will slow down calculation and may yield subpar results


100%|██████████| 1/1 [00:00<00:00, 78.86it/s]


WARNING! Your documents have duplicate entries!  This will slow down calculation and may yield subpar results


100%|██████████| 1/1 [00:00<00:00, 58.82it/s]


WARNING! Your documents have duplicate entries!  This will slow down calculation and may yield subpar results


100%|██████████| 1/1 [00:00<00:00, 79.14it/s]


In [ ]:
df.to_csv('hallucination_statistics.csv')